In [48]:
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

# Enter your prompt and speaker here
text_prompt = "[laughs] I'm AI and I'm here for realz, yo! Im gonna impact the world fast. You can't keep up already, but it's gonna get really fast after next month!"
voice_name = "AC" # use your custom voice name here if you have one

# load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [49]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True, 
    coarse_use_small=True,
    fine_use_gpu=True, 
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False,
    path="models"
)

No GPU being used. Careful, inference might be very slow!


In [53]:
%env  GLOBAL_ENABLE_MPS = True
%env  SUNO_ENABLE_MPS = True
# simple generation
audio_array = generate_audio(text_prompt, history_prompt=voice_name, text_temp=0.7, waveform_temp=0.7)

env: GLOBAL_ENABLE_MPS=False
env: SUNO_ENABLE_MPS=False










































































































































































































































































































































































100%|██████████| 100/100 [00:29<00:00,  3.41it/s]


AssertionError: 

In [ ]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_name,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

In [45]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [39]:
from scipy.io.wavfile import write as write_wav
# save audio
filepath = "audio.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)